In [8]:
# This block was stripped (and adapted) from "semeval_mapper.ipynb"
import hashlib
import matplotlib.pyplot as plt

def get_range():
    nof_elements = 500
    base_idx = (abs(int(hashlib.sha512("D'Amato".encode('utf-8')).hexdigest(), 16)) % 10)
    idx_intervallo = base_idx * 50+1
    return (idx_intervallo, idx_intervallo+50-1)

In [9]:
get_range()

(301, 350)

### Consegna 1: annotazione
si veda ./data/damato_semeval.tsv

### Consegna 1: valutazione

In [10]:
#from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import pandas as pd
from operator import add
import common.utils as utils
from itertools import product
import common.prettyprint as pp
from scipy.stats.stats import pearsonr, spearmanr
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import cohen_kappa_score, accuracy_score

La seguente funzione parte da una stringa, la divide in campi di testo (dividendo su `'\n'`) e restituisce una tupla `(campo_0, campi[1:])`. Questo serve a rendere il risultato, compatibile per essere trasformato in un dizionario.

In [11]:
def dictify(string):
    result = string.split('\n')
    return (result[0], result[1:])

In modo simile alla precedente, questa funzione serve per rendere la rappresentazione a stringhe compatibile con un Pandas DataFrame. I dati originali sono fatti così: `bnid__term [floats]`. Per renderlo compatibile, divide il primo campo su `'__'` (in modo da ottenerne due: l'id e il termine) e copia il resto del vettore così come sta.

In [12]:
def nasari_dfify(string):
    fields = string.split('\t')
    identifier = tuple(fields[0].split('__'))
    return utils.flatten([identifier, fields[1:]])

recupera i sensi dai vari termini. Il formato originale è:

```
#term
bnid1
bnid2
...
#termn
...
```
Quindi ho pensato di usare un dizionario, dal momento che la chiave è chiara (il termine), e i valori sono i synsetid. Per farlo, divido il contenuto del file sul carattere `'#'`, dal risultato rimuovo gli eventuali campi vuoti (`''`), il risultato viene passato a `dictify`, per ottenere una rappresentazione chiave-valore. Dopodiché il tutto viene convertito in dizionario python.

In [13]:
contents = open('./data/semeval17it_senses2synsets.txt', 'r').read()

cts = map(
        dictify,
        filter(
            lambda i: i != '' ,
            contents.split('#')
        )
)

semeval = dict(cts)

In modo simile a quanto fatto poco sopra, si legge il contenuto del file con i vettori nasari, si divide sul carattere (`'\n'`) per ottenere le varie linee del testo, si passa ognuna di queste linee a `'nasari_dfify'` per ottenere una rappresentazione che si presta a diventare un DataFrame, si converte e si rimuovono gli elementi vuoti (`''`).

In [14]:
nasari_cts = open('./data/mini_NASARI.tsv', 'r').read()
nasari = pd.DataFrame([tuple(nasari_dfify(line)) for line in nasari_cts.split('\n')])
nasari = nasari.dropna()

In [15]:
nasari

,0,1,2,3,4,5,6,7,8,9,...,292,293,294,295,296,297,298,299,300,301
0,bn:00023437n,CPU_card,-0.06913958,-0.06749106,0.00770207,0.12257188,-0.04002981,0.08145414,0.02884751,-0.03814174,...,-0.00691138,0.10257629,0.04694301,0.0665047,0.04471305,-0.12000545,-0.11998469,-0.01839009,-0.0928486,-0.01870047
1,bn:00050954n,Liberty,0.15167512,-0.01135582,0.08258358,0.27535351,-0.0413626,-0.04867816,0.12733052,0.04016196,...,-0.11470726,-0.10831937,-0.02482744,-0.05774688,-0.05825962,-0.03152054,0.08581172,-0.03614111,0.06264982,0.05308474
2,bn:02123809n,German_Freedom_Party,0.08157571,-0.02776888,-0.00129616,0.04452392,-0.1303027,-0.11249813,0.00266215,-0.12338018,...,-0.11501342,0.04564969,-0.04006878,0.0290099,-0.1000364,0.03304387,0.03676865,0.03205152,-0.00477991,0.09019229
3,bn:00007389n,Autonomy,0.05389084,-0.0789205,0.05398679,0.22686198,-0.03510364,-0.03242612,0.11709342,-0.11584815,...,-0.18894876,0.03867937,-0.09860891,-0.04987623,-0.09867302,0.05695884,-0.01611297,-0.00480201,0.07294346,0.06635249
4,bn:16819248n,Freedom_(Jonathan_Franzen_novel),0.0937584,-0.03049463,-0.05153185,0.21524077,0.0005526,-0.06747429,0.02467384,0.07155293,...,-0.06150085,0.00887872,-0.16739395,0.03010294,-0.06090382,-0.08805161,0.06766409,-0.00243878,-0.01704794,0.01385675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4555,bn:03629355n,Stargate_(device),0.10437215,0.05786481,0.01566665,0.04252388,-0.08804223,0.07409825,0.0136266,-0.10162532,...,-0.02099106,0.07335196,-0.13164267,0.03385111,-0.04264679,-0.0244316,-0.06918644,-0.027286,0.03342617,0.02678826
4556,bn:01528160n,Italian_submarine_Iride,0.02601548,0.14559146,-0.06802818,-0.03286261,-0.01270645,-0.04694423,-0.06491838,-0.1193463,...,-0.01706467,-0.01599527,-0.13512072,0.03791645,-0.01897756,-0.10180113,-0.17562765,0.00569892,0.11274927,-0.00583672
4557,bn:00018096n,Chemical_substance,-0.00140006,0.10361435,0.09291723,0.00541865,-0.17686922,-0.0343658,-0.0422839,-0.17321674,...,-0.10855723,-0.03284013,-0.04006074,0.0879663,-0.12378977,-0.0496634,-0.03910206,0.07541048,0.1075351,-0.0093504
4558,bn:02345076n,Le_Cadeau,-0.00787666,0.09724722,0.04336747,0.10713132,0.08558352,0.05982183,-0.00096916,-0.15680629,...,-0.11233576,0.03459905,-0.14523473,0.07472088,0.08261698,-0.02326385,-0.06596823,-0.00155704,-0.0383024,0.00116444


La prossima funzione prende il vettore nasari corrispondente ad un babelnet synsetid, cercandolo sul DataFrame creato appena sopra.

In [16]:
def get_vector_for_id(bnid):
    return np.array(nasari[nasari[0] == bnid].values[0][2:]).reshape(1, -1)

La prossima funzione calcola il massimo valore di cosine similarity dati due termini. Gli viene passata una Series (Pandas) che corrisponde ad una tupla del file `./data/damato_semeval.tsv`. Quindi cerca sul dizionario `semval` costruito sopra, tutti i synsetid relativi ai due termini. Per ognuno di essi, prende da `nasari` i relativi vettori (sfruttando `get_vector_for_id`). Ne calcola la cosine similarity e la aggiunge a tutti gli score. Quando ha finito di esplorare tutte le alternative, restituisce il massimo degli score.

In [17]:
def get_cosine_score(series):
    #print(series)
    try:
        bnidf = list(filter(lambda item: item != '', semeval[series[0]]))
        bnids = list(filter(lambda item: item != '', semeval[series[1]]))
        #print(bnidf, bnids)
        combos = list(product(bnidf, bnids))
        scores = [0]
        for combo in combos:
            try:
                scores.append(cosine_similarity(get_vector_for_id(combo[0]), get_vector_for_id(combo[1]))[0][0])
            except IndexError as e:
                pass
                #print(f"Combo not found: {combo}, skipping...")
        return max(scores)
    except KeyError:
        return None

La prossima funzione è una specializzazione della precedente, con l'eccezione che quando ha finito di esplorare le possibili combinazioni, trova l'indice dello score massimo, e restituisce la combinazione presente su quell'indice. (Salta un elemento perché `max` non può essere valutata su una lista vuota, dovendo avere almeno un elemento aggiungo '0'). In caso trovi errori (le combinazioni non esistono), oppure i termini non esistono sul dizionario `semeval`, restituisce `("N/A", "N/A")`

In [53]:
def get_argmax_cosine_score(series):
    #print(series)
    try:
        bnidf = list(filter(lambda item: item != '', semeval[series[0]]))
        bnids = list(filter(lambda item: item != '', semeval[series[1]]))
        #print(bnidf, bnids)
        combos = list(product(bnidf, bnids))
        scores = [0]
        for combo in combos:
            try:
                scores.append(cosine_similarity(get_vector_for_id(combo[0]), get_vector_for_id(combo[1]))[0][0])
            except IndexError as e:
                pass
        if len(combos) > 0:
            # -1 to account for the extra '0' in the scores
            selectedcombo = combos[scores.index(max(scores))-1]
            return selectedcombo[0], selectedcombo[1]
        else:
            return "N/A","N/A"
    except KeyError:
        return "N/A","N/A"

In [61]:
damato_semeval = pd.read_csv('./data/damato_semeval.tsv', sep='\t')

In [55]:
cosim = damato_semeval.apply(lambda s: get_cosine_score(s), axis=1)

Calcola le correlazioni tra gli score annotati a mano e il massimo calcolato massimizzando la cosine similarity. Sfrutta `pearsonr` e `spearmanr` da `scipy`.

In [56]:
print(f"{pp.success(pp.bold('Pearson Correlation: '))}: {pearsonr(damato_semeval.score, cosim)[0]}")
print(f"{pp.success(pp.bold('Spearman Correlation'))}: {spearmanr(damato_semeval.score, cosim)[0]}")



Pearson Correlation: : 0.48035487776530833
Spearman Correlation: 0.4754073018375166


### Consegna 2: Sense Identification
Formato dell'annotazione:  
#Term1 Term2 BS1 BS2 Terms_in_BS1 Terms_in_BS2

file: `./data/damato_semeval.tsv`

### Consegna 2: Agreement nell'annotazione

**NOTA**: Pandas tenta di inferire il tipo del dato quando legge un file. Questo lo porta a volte a leggere i babelnet synset id come float. Per evitare che questo accada è necessario forzarne il tipo ad essere `str`. per questo motivo, nei punti in cui sono coinvolti `damato_semeval.BS1` e `damato_semeval.BS2`, li uso come `damato_semeval.*.apply(str)`, che scorre tutto il vettore e applica `str`, ad ogni elemento, convertendolo in stringa.

In [58]:
argmax = damato_semeval.apply(lambda s: get_argmax_cosine_score(s), axis=1)
bs1, bs2 = zip(*argmax)

In [76]:
print(f"Agreement per BS1: {cohen_kappa_score(bs1, damato_semeval.BS1.apply(str))}")
print(f"Agreement per BS2: {cohen_kappa_score(bs2, damato_semeval.BS2.apply(str))}")

Agreement per BS1: 0.4150867285195643
Agreement per BS2: 0.3156199677938808


### Consegna 2: Valutazione dell'annotazione

Per calcolare l'accuratezza sulla tupla in generale, unisco le stringhe trattandole come label singole.

In [77]:
print(f"Accuratezza su BS1: {accuracy_score(damato_semeval.BS1.apply(str), bs1)}")
print(f"Accuratezza su BS2: {accuracy_score(damato_semeval.BS2.apply(str), bs2)}")
damato_tuples = [item[0] + item[1] for item in zip(damato_semeval.BS1.apply(str), damato_semeval.BS2.apply(str))]
argmax_tuples = [item[0] + item[1] for item in argmax]
print(f"Accuratezza su BS1+BS2: {accuracy_score(damato_tuples, argmax_tuples)}")



Accuratezza su BS1: 0.42
Accuratezza su BS2: 0.32
Accuratezza su BS1+BS2: 0.16
